In [1]:
#import libraries
import requests
import pandas as pd
import time
import psycopg2 as ps
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
API_KEY = 'xxxxxx' # Find on Slickremix how to get key
CHANNEL_ID = 'xxxxx'

In [3]:
def get_video_details(video_id):
    
    # collecting view, like, dislike, comments count
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
    response_video_stats = requests.get(url_video_stats).json()
        
    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    #dislike_count = response_video_stats['items'][0]['statistics']['dislikeCount'] private as of December 13, 2021
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']
    
    return view_count, like_count, comment_count

In [4]:
def get_videos(df):
    pageToken = ""
    while 1:
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken

        response = requests.get(url).json()
        time.sleep(3) #give it a few second before starting the for loop
        
        
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&amp;","")
                video_title = str(video_title).replace("&#39;", "")
                video_title = str(video_title).replace("&", "")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                view_count, like_count, comment_count = get_video_details(video_id)

                df = df.append({'video_id':video_id,'video_title':video_title,
                                "upload_date":upload_date,"view_count":view_count,
                                "like_count":like_count,"comment_count":comment_count},ignore_index=True)
        try:
            if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break


    return df

In [5]:
# build a dataframe
df = pd.DataFrame(columns=["video_id", "video_title", "upload_date", "view_count", "like_count", "comment_count"])

df = get_videos(df)

In [6]:
df

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,GM-OI7HcCeU,Why 50% of Indians Live North of This Line,2022-11-22,719133,27545,3579
1,lp0Sxn42TGs,Why Russia Destroyed the Worlds 4th Biggest Lake,2022-11-08,2077001,48734,5613
2,f1WFrsz4g14,How War in Ukraine is Destroying Russia,2022-10-31,3860367,121482,14204
3,jjOW6kLEckg,How Libya Built Brand-New Rivers Across the Sa...,2022-10-20,1552236,40307,3167
4,7NOMj7n6QAM,Why Azerbaijan Will Keep Attacking Armenia,2022-10-15,1800858,64755,10762
...,...,...,...,...,...,...
295,9ZZy594n9hY,Brexit Explained: What Could Happen to the UK ...,2016-07-14,756546,11342,2894
296,Pj9fn4L4Njk,The Real Life Game of Thrones Part 1: Is Great...,2016-05-26,2106245,37386,4485
297,AsxBalxAqzg,What Does the Spanish Flag Mean?,2016-05-01,1985821,34948,7635
298,PdFt7TOWq0w,What Does the French Flag Mean?,2016-04-08,2577396,43757,9285


In [8]:
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)
    
    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
    return conn

In [10]:
# create table

def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
                    video_id VARCHAR(255) PRIMARY KEY,
                    video_title TEXT NOT NULL,
                    upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                    view_count INTEGER NOT NULL,
                    like_count INTEGER NOT NULL,
                    comment_count INTEGER NOT NULL
            )""")
    
    curr.execute(create_table_command)

In [21]:
#check if video exists
def check_if_video_exists(curr, video_id):
    query = ("""SELECT video_id FROM VIDEOS WHERE video_id = %s""")
    curr.execute(query, (video_id,))
    
    return curr.fetchone() is not None # Returns a single row from the table

In [22]:
#update row if video exists

def update_row(curr, video_id, video_title, view_count, like_count, comment_count):
    query = ("""UPDATE videos
              SET video_title = %s,
                view_count = %s,
                like_count = %s,
                comment_count = %s
              WHERE video_id = %s;""")
    vars_to_update = (video_title, view_count, like_count, comment_count, video_id)
    curr.execute(query, vars_to_update)

In [23]:
#check to see if video exists

def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count',
                                   'like_count', 'comment_count'])
    for i, row in df.iterrows():
        if check_if_video_exists(curr, row['video_id']): # If video already exists then we will update
            update_row(curr,row['video_id'],row['video_title'],row['view_count'],row['like_count'],row['comment_count'])
        else: # The video doesn't exists so we will add it to a temp df and append it using append_from_df_to_db
            tmp_df = tmp_df.append(row)

    return tmp_df

In [29]:
#write insert command

def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, comment_count):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date, view_count, like_count, comment_count)
    VALUES(%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (video_id, video_title, upload_date, view_count, like_count, comment_count)
    curr.execute(insert_into_videos, row_to_insert)

In [30]:
def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'],
                          row['view_count'], row['like_count'], row['comment_count'])

In [31]:
#database credentials
host_name = 'xxxxxx.rds.amazonaws.com'
dbname = 'xxxx'
port = '5432'
username = 'xxxx'
password = 'xxxxxxxxx'
conn = None

conn = connect_to_db(host_name, dbname, port, username, password)

Connected!


In [32]:
curr = conn.cursor()

In [13]:
create_table(curr) # Only neeeds to be used once

In [33]:
# Update the database table
new_vid_df = update_db(curr, df)

In [35]:
# Insert new videos into database table
append_from_df_to_db(curr, new_vid_df)

In [36]:
conn.commit()

In [ ]:
# Create a scheduling job to run every week

""" 
First step is to create a batch file. Open notepad and save a file with .bat extension with your paths:

"C:\Program Files\Python37\python.exe"
"C:\your_custom_path\python_file_name.py"
pause

Activate the scheduler:

1.Open Windows Task Scheduler

2.In the Action tab choose Create Basic Task

3.Click Next and choose the time you want the script to run

4.Click Next and in the program/script box enter the path to your python.exe

5.In the Add Arguments box enter the name of the python file you’re scheduling

6.In the Start In box enter the path the to python file

"""